Normalize Predictions Data

In [37]:
from BHDVCS_tf import DvcsData
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("dvcs_bkm_xs_June2021_4pars.csv")
errValues = [0.0, 0.01, 0.025, 0.05, 0.1, 0.15, 0.25, 0.30]

def getReals(setNum, numPhi = 45):
    df = pd.read_csv("dvcs_bkm_xs_June2021_4pars.csv")
    return df.loc[numPhi * setNum, 'gReH': 'gReHTilde']

def normalize(setNum):
    for err in errValues: 
            #Getting data from the file
            data = pd.read_csv("/home/knc8xp/Predictions/Method2-"+ str(setNum) + "-" + str(err) +  ".csv")

            #Gets Real values
            HReal, EReal, tildeReal = getReals(setNum, numPhi=45)

            ReHMean = np.mean(data.ReH)
            ReEMean = np.mean(data.ReE)
            ReHtildeMean = np.mean(data.ReHtilde)

            ReHSTD = np.std(data.ReH)
            ReESTD = np.std(data.ReE)
            ReHtildeSTD = np.std(data.ReHtilde)

            data['ReH'] -= HReal
            data['ReE'] -= EReal
            data['ReHtilde'] -= tildeReal

            data['ReH'] /= HReal
            data['ReE'] /= EReal
            data['ReHtilde'] /= tildeReal

            data.to_csv("Predictions/Method2-"+ str(setNum) + "-" + str(err) +  ".csv")

def getColorMappingValues(errF, amountofSets):
    ReHMean = []
    ReEMean = []
    ReHtildeMean = []
    for set in range(amountofSets):
        data = pd.read_csv("Predictions\Method2-"+ str(set) + "-" + str(errF) +  ".csv")
        
        # NOTE we take max(mean, -1) because if data is more than 100% error it should just cap in terms of heatmapping
        #Inverted because currently smaller values are red and larger values are green
        #Want to make opposite (smaller %error-values = green and larger %error-values = red) therefore we multiply by -1
        ReHMean.append(100 * max(-abs(np.mean(data.ReH)), -1))
        ReEMean.append(100 * max(-abs(np.mean(data.ReE)), -1))
        ReHtildeMean.append(100 * max(-abs(np.mean(data.ReHtilde)), -1))

    return ReHMean, ReEMean, ReHtildeMean 

#Returns lists that contain the kinematics for each index
def getkinematics(numSets):
    k = []
    QQ = []
    xb = []
    t = []
    for i in range(numSets):
        #gets a set's kinematics
        kvar, Q, x, ts = df.loc[45 * i, 'k': 't']
        
        #adds set's kinematics to running list
        k.append(kvar)
        QQ.append(Q)
        xb.append(x)
        t.append(ts)
    
    return k, QQ, xb, t

import plotly.graph_objects as go
import plotly.express as px

def plotInteractive(err, numSets, whichCFF, show = True):
    h, e, tilde = getColorMappingValues(err, numSets)
    _, QQ, xb, t = getkinematics(numSets)

    chosen = []
    if whichCFF == 'ReH':
        chosen = h
    elif whichCFF == 'ReE':
        chosen = e
    else: 
        chosen = tilde
        whichCFF = 'ReHtilde'

    size = [6] * numSets

    # Blue means better predictions -- Red --> Worse Predictions
    splot = go.Figure(data=[go.Scatter3d(
        x=QQ,
        y=t,
        z=xb,
        mode='markers',
        marker=dict(
            size=5,
            color=chosen,                # set color to an array/list of desired values
            colorscale='RdYlGn',   # choose a colorscale
            opacity=0.8
        )
    )])

    # splot = px.scatter_3d(x=QQ, y=t, z=xb,
    #                 color=chosen, color_continuous_scale = "RdYlGn", size=size)


    splot.update_layout(
        title = 'Prediction Accuracy on ' + whichCFF + ' with ' + str(err*100) + '% Error on F',
        scene = dict(
            xaxis_title='QQ',
            yaxis_title='x_b',
            zaxis_title='t',
            aspectmode='cube',
        ),
        template='ggplot2',
        margin=dict(r=0, b=40, l=0, t=50)
    )

    splot.write_html('Interactive Plots/Mega/Pred%Acc_' + whichCFF + '_' +str(err*100) + 'ErrF.html')

    if show:
        splot.show()

# Mega 3D Plots With All Sets

In [38]:
plotInteractive(0.0, 403, 'ReH')

In [39]:
plotInteractive(0.1, 403, 'ReH')


In [43]:
plotInteractive(0.25, 403, 'ReH')

# Create All Plots

In [ ]:
errors = [0.0, 0.01, 0.025, 0.05, 0.1, 0.15, 0.25, 0.30]
for err in errors: 
    plotInteractive(err, 403, 'ReH')
    plotInteractive(err, 403, 'ReE')
    plotInteractive(err, 403, 'ReHtilde')